In [1]:
from keras.datasets import cifar10
from keras.utils import to_categorical
import keras

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train / 255
x_test = x_test / 255
y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_test = keras.utils.to_categorical(y_test, num_classes=10)

Using TensorFlow backend.


In [2]:
x_train.shape

(50000, 32, 32, 3)

首先，我们来定义conv1，即第一层的卷积层与最大池化层。注意，这里我们不再采用Sequential模型，而是采用Functional模型，Functional模型方便我们定义上文提到的ResNet的核心残缺模块。

首先定义我们的第一层即conv1，Functional模型首先需要根据我们的输入数据定义一个Input，我们的图片大小是32*32*3，所以参数shape对应的值是32,32,3，接下来这里我们添加了一个ZeroPadding2D层，正如其名，改层能给图片在边缘补零，因为我们的图片较小，这里我们添加一个3*3的ZeroPadding2D层，之后就是我们比较熟悉的卷积层了，卷积层之后我们添加了BatchNormalization层，BatchNormalization能加速收敛，控制过拟合并允许使用较大的学习率，参数axis指需要规范化的轴，通常为特征轴，最后再加上激活函数与最大池化层。

In [3]:
import numpy as np
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D, add, Input, ZeroPadding2D, BatchNormalization, Activation, AveragePooling2D

input_data = Input(shape=(32,32,3))
x = ZeroPadding2D((3, 3))(input_data)
x = Conv2D(64, (2, 2), strides=(1, 1))(x)
x = BatchNormalization(axis=3)(x)
x = Activation('relu')(x)
x = MaxPooling2D((3, 3), strides=(2, 2))(x)

接下来我们定义残缺模块，这里我们封装成一个方法方便后面调用。注意，我们在把卷积之后的结果和原数据进行相加前，给原数据添加了卷积层，可以看到，其卷积核的大小是1*1，所以其目的主要是为了改变原数据的维度，从而能让数据进行相加的操作。

In [19]:
def identity_Block(input_data, n_filter, kernel_size=(3, 3), strides=(1, 1)):
    x = Conv2D(n_filter, kernel_size, padding='same', strides=strides)(input_data)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)

    x = Conv2D(n_filter, kernel_size, padding='same', strides=strides)(x)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)

    shortcut = Conv2D(n_filter, kernel_size=(1, 1), padding='same')(input_data)
    shortcut = BatchNormalization(axis=3)(shortcut)

    x = add([x, shortcut])
    x = Activation('relu')(x)
    return x

定义conv2_x到conv5_x的卷积层

In [20]:
x = identity_Block(x, n_filter=64)
x = identity_Block(x, n_filter=64)

x = identity_Block(x, n_filter=128)
x = identity_Block(x, n_filter=128)

x = identity_Block(x, n_filter=256)
x = identity_Block(x, n_filter=256)

x = identity_Block(x, n_filter=512)
x = identity_Block(x, n_filter=512)

添加平均池化层与sofxmax层。

In [21]:
from keras.models import Model

x = AveragePooling2D(pool_size=(3, 3))(x)
x = Flatten()(x)
x = Dense(10, activation='softmax')(x)

model = Model(inputs=input_data, outputs=x)

In [22]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 37, 37, 64)   832         zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 37, 37, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation

添加early_stopping并进行数据训练与测试。

In [23]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='acc', patience=3, verbose=1)
model.fit(x_train, y_train, batch_size=64, epochs=200, callbacks=[early_stopping])

Epoch 1/200
50000/50000 [==============================] - 342s 7ms/step - loss: 2.3734 - acc: 0.2878
Epoch 2/200
50000/50000 [==============================] - 329s 7ms/step - loss: 1.9178 - acc: 0.3842
Epoch 3/200
50000/50000 [==============================] - 337s 7ms/step - loss: 1.4367 - acc: 0.4803
Epoch 4/200
50000/50000 [==============================] - 329s 7ms/step - loss: 1.2668 - acc: 0.5468
Epoch 5/200
50000/50000 [==============================] - 328s 7ms/step - loss: 1.1136 - acc: 0.6060
Epoch 6/200
50000/50000 [==============================] - 328s 7ms/step - loss: 0.9788 - acc: 0.6566
Epoch 7/200
50000/50000 [==============================] - 328s 7ms/step - loss: 0.8637 - acc: 0.6987
Epoch 8/200
50000/50000 [==============================] - 328s 7ms/step - loss: 0.7715 - acc: 0.7310
Epoch 9/200
50000/50000 [==============================] - 328s 7ms/step - loss: 0.6917 - acc: 0.7579
Epoch 10/200
50000/50000 [==============================] - 328s 7ms/step - loss: 

In [ ]:
score = model.evaluate(x_test, y_test, batch_size=64)
print('Test accuracy:', score[1])